<a href="https://colab.research.google.com/github/kridtapon/Future-Trend-Crossover/blob/main/Future_Trend_Crossover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Displaced Moving Averages (DMA)
def calculate_dma(df, period, shift):
    return df['Close'].rolling(window=period).mean().shift(shift)

# Define the stock symbol and time period
symbol = 'NVDA'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate DMAs
df['DMA_fast'] = calculate_dma(df, period=50, shift=10)
df['DMA_slow'] = calculate_dma(df, period=200, shift=20)

# Define Entry and Exit signals
df['Entry'] = df['DMA_fast'] > df['DMA_slow']
df['Exit'] = df['DMA_fast'] < df['DMA_slow']

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)


Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           1937284.73099
Total Return [%]                      1837.284731
Benchmark Return [%]                  2148.392946
Max Gross Exposure [%]                      100.0
Total Fees Paid                        728.704914
Max Drawdown [%]                        46.567515
Max Drawdown Duration                       374.0
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                     1622882.638578
Win Rate [%]                                100.0
Best Trade [%]                         214.616495
Worst Trade [%]                        214.616495
Avg Winning Trade [%]                  214.616495
Avg Losing Trade [%]                          NaN
